In [1]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandasql import sqldf
import numpy as np
import datetime
import waterfall_chart

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import log_loss
from sklearn.dummy import DummyClassifier

# For demonstrative pruposes
from scipy.special import logit, expit

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV,\
cross_val_score, RandomizedSearchCV

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, \
ExtraTreesClassifier, VotingClassifier, StackingRegressor, StackingClassifier

from sklearn.naive_bayes import MultinomialNB, GaussianNB

In [2]:
df = pd.read_csv('../Data/monthly_full_release_long_format.csv', parse_dates=['Date'])

In [3]:
df.head(50)

,Area,Country code,Date,Area type,Continent,Ember region,EU,OECD,G20,G7,Category,Subcategory,Variable,Unit,Value,YoY absolute change,YoY % change
0,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity demand,Demand,Demand,TWh,12.77,NaN,NaN
1,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,Clean,%,34.57,NaN,NaN
2,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,Fossil,%,65.44,NaN,NaN
3,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,Gas and Other Fossil,%,63.40,NaN,NaN
4,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,"Hydro, Bioenergy and Other Renewables",%,29.08,NaN,NaN
5,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,Renewables,%,29.55,NaN,NaN
6,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,Wind and Solar,%,0.47,NaN,NaN
7,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,Clean,TWh,4.41,NaN,NaN
8,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,Fossil,TWh,8.35,NaN,NaN
9,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,Gas and Other Fossil,TWh,8.09,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 329776 entries, 0 to 329775
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Area                 329776 non-null  object        
 1   Country code         300776 non-null  object        
 2   Date                 329776 non-null  datetime64[ns]
 3   Area type            329776 non-null  object        
 4   Continent            300776 non-null  object        
 5   Ember region         300776 non-null  object        
 6   EU                   300776 non-null  float64       
 7   OECD                 300776 non-null  float64       
 8   G20                  300776 non-null  float64       
 9   G7                   300776 non-null  float64       
 10  Category             329776 non-null  object        
 11  Subcategory          329776 non-null  object        
 12  Variable             329776 non-null  object        
 13  Unit          

In [5]:
for x in list(df.columns):
    print(df[x].value_counts())

Australia                   12496
United States of America    12372
Azerbaijan                   8000
Belarus                      8000
Armenia                      8000
                            ...  
Costa Rica                   1476
Georgia                      1344
Taiwan                       1320
Burundi                      1170
Kosovo                        600
Name: Area, Length: 85, dtype: int64
AUS    12496
USA    12372
ARM     8000
BLR     8000
AZE     8000
       ...  
CRI     1476
GEO     1344
TWN     1320
BDI     1170
XKX      600
Name: Country code, Length: 76, dtype: int64
2021-09-01    4044
2021-08-01    4044
2021-10-01    4018
2021-11-01    4013
2020-10-01    3994
              ... 
1999-10-01      44
1999-08-01      44
1999-06-01      44
1999-01-01      44
1999-09-01      44
Name: Date, Length: 284, dtype: int64
Country    300776
Region      29000
Name: Area type, dtype: int64
Europe           155649
Asia              56342
Africa            29091
North America   

In [6]:
list(df.columns)

['Area',
 'Country code',
 'Date',
 'Area type',
 'Continent',
 'Ember region',
 'EU',
 'OECD',
 'G20',
 'G7',
 'Category',
 'Subcategory',
 'Variable',
 'Unit',
 'Value',
 'YoY absolute change',
 'YoY % change']

In [7]:
df.Area.value_counts()

Australia                   12496
United States of America    12372
Azerbaijan                   8000
Belarus                      8000
Armenia                      8000
                            ...  
Costa Rica                   1476
Georgia                      1344
Taiwan                       1320
Burundi                      1170
Kosovo                        600
Name: Area, Length: 85, dtype: int64

In [8]:
df['Area type'].value_counts()

Country    300776
Region      29000
Name: Area type, dtype: int64

In [9]:
df = df.loc[df['Area type'] == 'Country'].copy()

In [10]:
df.isna().sum()

Area                        0
Country code                0
Date                        0
Area type                   0
Continent                   0
Ember region                0
EU                          0
OECD                        0
G20                         0
G7                          0
Category                    0
Subcategory                 0
Variable                    0
Unit                        0
Value                       0
YoY absolute change    119067
YoY % change           165619
dtype: int64

In [11]:
df.drop(columns=['YoY absolute change', 'YoY % change'], inplace=True)

df

,Area,Country code,Date,Area type,Continent,Ember region,EU,OECD,G20,G7,Category,Subcategory,Variable,Unit,Value
0,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity demand,Demand,Demand,TWh,12.77
1,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,Clean,%,34.57
2,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,Fossil,%,65.44
3,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,Gas and Other Fossil,%,63.40
4,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,"Hydro, Bioenergy and Other Renewables",%,29.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327721,Viet Nam,VNM,2022-06-01,Country,Asia,Asia,0.0,0.0,0.0,0.0,Power sector emissions,Fuel,Hydro,mtCO2,0.27
327722,Viet Nam,VNM,2022-06-01,Country,Asia,Asia,0.0,0.0,0.0,0.0,Power sector emissions,Fuel,Other Fossil,mtCO2,0.00
327723,Viet Nam,VNM,2022-06-01,Country,Asia,Asia,0.0,0.0,0.0,0.0,Power sector emissions,Fuel,Solar,mtCO2,0.12
327724,Viet Nam,VNM,2022-06-01,Country,Asia,Asia,0.0,0.0,0.0,0.0,Power sector emissions,Fuel,Wind,mtCO2,0.01


In [12]:
df.head()

,Area,Country code,Date,Area type,Continent,Ember region,EU,OECD,G20,G7,Category,Subcategory,Variable,Unit,Value
0,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity demand,Demand,Demand,TWh,12.77
1,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,Clean,%,34.57
2,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,Fossil,%,65.44
3,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,Gas and Other Fossil,%,63.40
4,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,"Hydro, Bioenergy and Other Renewables",%,29.08


In [13]:
df.isna().sum()

Area            0
Country code    0
Date            0
Area type       0
Continent       0
Ember region    0
EU              0
OECD            0
G20             0
G7              0
Category        0
Subcategory     0
Variable        0
Unit            0
Value           0
dtype: int64

In [14]:
df.Area.value_counts()

Australia                   12496
United States of America    12372
Azerbaijan                   8000
Armenia                      8000
Belarus                      8000
                            ...  
Costa Rica                   1476
Georgia                      1344
Taiwan                       1320
Burundi                      1170
Kosovo                        600
Name: Area, Length: 76, dtype: int64

In [15]:
df.head()

,Area,Country code,Date,Area type,Continent,Ember region,EU,OECD,G20,G7,Category,Subcategory,Variable,Unit,Value
0,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity demand,Demand,Demand,TWh,12.77
1,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,Clean,%,34.57
2,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,Fossil,%,65.44
3,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,Gas and Other Fossil,%,63.40
4,Argentina,ARG,2018-01-01,Country,South America,Latin America and Caribbean,0.0,0.0,1.0,0.0,Electricity generation,Aggregate fuel,"Hydro, Bioenergy and Other Renewables",%,29.08


In [16]:
df['Date_year'] = df.Date.dt.year.astype(float)

df['Date_year']

0         2018.0
1         2018.0
2         2018.0
3         2018.0
4         2018.0
           ...  
327721    2022.0
327722    2022.0
327723    2022.0
327724    2022.0
327725    2022.0
Name: Date_year, Length: 300776, dtype: float64

In [17]:
df['Date_month'] = df.Date.dt.month.astype(float)
df['Date_month']

0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
327721    6.0
327722    6.0
327723    6.0
327724    6.0
327725    6.0
Name: Date_month, Length: 300776, dtype: float64

In [18]:
relevant_col = ['Date_year', 'Date_month','Ember region',
                'EU', 'OECD', 'G20', 'G7',
                'Category', 'Subcategory','Variable',
                'Value']

In [19]:
X = df[relevant_col]
y = df['Area']

In [20]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300776 entries, 0 to 327725
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Date_year     300776 non-null  float64
 1   Date_month    300776 non-null  float64
 2   Ember region  300776 non-null  object 
 3   EU            300776 non-null  float64
 4   OECD          300776 non-null  float64
 5   G20           300776 non-null  float64
 6   G7            300776 non-null  float64
 7   Category      300776 non-null  object 
 8   Subcategory   300776 non-null  object 
 9   Variable      300776 non-null  object 
 10  Value         300776 non-null  float64
dtypes: float64(7), object(4)
memory usage: 27.5+ MB


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42)

In [22]:
X_train_nums = X_train[['Date_year', 'Date_month', 'Value']]

ss = StandardScaler()

ss.fit(X_train_nums)
nums_df = pd.DataFrame(ss.transform(X_train_nums),
                      index=X_train_nums.index)

In [23]:
X_train_cat = X_train.select_dtypes('object')

ohe = OneHotEncoder(
    drop='first',
    sparse=False)

dums = ohe.fit_transform(X_train_cat)
dums_df = pd.DataFrame(dums,
                       columns=ohe.get_feature_names(),
                       index=X_train_cat.index)

In [24]:
X_train_group = X_train[['EU', 'OECD', 'G20', 'G7']]

grouping_df = pd.DataFrame(X_train_group,
                           index=X_train_group.index)

In [25]:
X_train_clean = pd.concat([nums_df, dums_df, grouping_df], axis=1)

In [26]:
X_train_clean.columns

Index([                                         0,
                                                1,
                                                2,
                                        'x0_Asia',
                                      'x0_Europe',
                 'x0_Latin America and Caribbean',
                               'x0_North America',
                                     'x0_Oceania',
                      'x1_Electricity generation',
                         'x1_Electricity imports',
                          'x1_Electricity prices',
                      'x1_Power sector emissions',
                               'x2_CO2 intensity',
                 'x2_Day-ahead electricity price',
                                      'x2_Demand',
                         'x2_Electricity imports',
                                        'x2_Fuel',
                                       'x2_Total',
                               'x3_CO2 intensity',
                               

#### Preparing the Test Set

In [27]:
X_test_cat = X_test.select_dtypes('object')

test_dums = ohe.transform(X_test_cat)
test_dums_df = pd.DataFrame(test_dums,
                       columns=ohe.get_feature_names(),
                      index=X_test_cat.index)

In [28]:
X_test_nums = X_test[['Date_year', 'Date_month', 'Value']]

test_nums = ss.transform(X_test_nums)
test_nums_df = pd.DataFrame(test_nums,
                           index=X_test_nums.index)

In [29]:
X_test_group = X_test[['EU', 'OECD', 'G20', 'G7']]

grouping_df = pd.DataFrame(X_test_group,
                           index=X_test_group.index)

In [30]:
X_test_clean = pd.concat([test_nums_df,
                 test_dums_df, grouping_df], axis=1)

In [31]:
X_test_clean.head()

,0,1,2,x0_Asia,x0_Europe,x0_Latin America and Caribbean,x0_North America,x0_Oceania,x1_Electricity generation,x1_Electricity imports,...,x3_Renewables,x3_Solar,x3_Total Generation,x3_Total emissions,x3_Wind,x3_Wind and Solar,EU,OECD,G20,G7
102499,0.166758,0.197847,-0.254408,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
269462,0.901324,-0.092560,-0.274046,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
165952,-0.322954,-0.963782,-0.270259,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
181199,1.146180,-0.673375,5.658619,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
100130,-0.812665,0.197847,-0.274467,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
knn = KNeighborsClassifier()

knn.fit(X_train_clean, y_train)

KNeighborsClassifier()

In [33]:
knn.score(X_train_clean, y_train)

0.5391862688055856

In [34]:
knn.score(X_test_clean, y_test)

0.3503723651838553

##### Decreasing $k$

In [35]:
knn3 = KNeighborsClassifier(n_neighbors=3)

knn3.fit(X_train_clean, y_train)

KNeighborsClassifier(n_neighbors=3)

In [36]:
knn3.score(X_train_clean, y_train)

0.5571856038567036

In [37]:
knn3.score(X_test_clean, y_test)

0.3279805838154133

#### Decision Tree

In [38]:
dt = DecisionTreeClassifier(random_state=42)

dt.fit(X_train_clean, y_train)

DecisionTreeClassifier(random_state=42)

In [39]:
dt.score(X_train_clean, y_train)

0.8612501038982628

In [40]:
dt.score(X_test_clean, y_test)

0.6053261520047876

##### Changing the branching criterion

In [41]:
dtb = DecisionTreeClassifier(random_state=42, criterion='entropy')

dtb.fit(X_train_clean, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=42)

In [42]:
dtb.score(X_train_clean, y_train)

0.8612501038982628

In [43]:
dtb.score(X_test_clean, y_test)

0.606140700844471

# Looking at EU VS Non-EU

In [48]:
filter_EU = df['EU'] == 1
filter_no_EU = df['EU'] == 0

In [49]:
df_eu = df[filter_EU]
df_no_eu = df[filter_no_EU]

In [50]:
relevant_col = ['Date_year', 'Date_month','Ember region',
                'Category', 'Subcategory','Variable',
                'Value']

In [51]:
X_eu = df_eu[relevant_col]
y_eu = df_eu['Area']

In [52]:
X_no_eu = df_no_eu[relevant_col]
y_no_eu = df_no_eu['Area']

In [53]:
X_eu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114053 entries, 25134 to 281321
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Date_year     114053 non-null  float64
 1   Date_month    114053 non-null  float64
 2   Ember region  114053 non-null  object 
 3   Category      114053 non-null  object 
 4   Subcategory   114053 non-null  object 
 5   Variable      114053 non-null  object 
 6   Value         114053 non-null  float64
dtypes: float64(3), object(4)
memory usage: 7.0+ MB


### EU Models

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X_eu, y_eu, test_size=0.2, 
                                                    random_state=42)

In [55]:
X_train_nums = X_train[['Date_year', 'Date_month', 'Value']]

ss = StandardScaler()

ss.fit(X_train_nums)
nums_df = pd.DataFrame(ss.transform(X_train_nums),
                      index=X_train_nums.index)

In [56]:
X_train_cat = X_train.select_dtypes('object')

ohe = OneHotEncoder(
    drop='first',
    sparse=False)

dums = ohe.fit_transform(X_train_cat)
dums_df = pd.DataFrame(dums,
                       columns=ohe.get_feature_names(),
                       index=X_train_cat.index)

In [57]:
X_train_clean = pd.concat([nums_df, dums_df], axis=1)

In [58]:
X_train_clean.columns

Index([                                         0,
                                                1,
                                                2,
                      'x1_Electricity generation',
                         'x1_Electricity imports',
                          'x1_Electricity prices',
                      'x1_Power sector emissions',
                               'x2_CO2 intensity',
                 'x2_Day-ahead electricity price',
                                      'x2_Demand',
                         'x2_Electricity imports',
                                        'x2_Fuel',
                                       'x2_Total',
                               'x3_CO2 intensity',
                                       'x3_Clean',
                                        'x3_Coal',
                 'x3_Day-ahead electricity price',
                                      'x3_Demand',
                                      'x3_Fossil',
                               

#### Preparing the Test Set

In [59]:
X_test_cat = X_test.select_dtypes('object')

test_dums = ohe.transform(X_test_cat)
test_dums_df = pd.DataFrame(test_dums,
                       columns=ohe.get_feature_names(),
                      index=X_test_cat.index)

In [60]:
X_test_nums = X_test[['Date_year', 'Date_month', 'Value']]

test_nums = ss.transform(X_test_nums)
test_nums_df = pd.DataFrame(test_nums,
                           index=X_test_nums.index)

In [61]:
X_test_clean = pd.concat([test_nums_df,
                 test_dums_df], axis=1)

In [62]:
X_test_clean.head()

,0,1,2,x1_Electricity generation,x1_Electricity imports,x1_Electricity prices,x1_Power sector emissions,x2_CO2 intensity,x2_Day-ahead electricity price,x2_Demand,...,x3_Net Imports,x3_Nuclear,x3_Other Fossil,x3_Other Renewables,x3_Renewables,x3_Solar,x3_Total Generation,x3_Total emissions,x3_Wind,x3_Wind and Solar
89245,0.696663,-0.084918,-0.270622,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107915,-0.699857,-0.376507,-0.274250,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59441,-0.699857,0.206671,-0.273677,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
228013,-1.165363,0.789849,-0.106037,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
230911,1.162169,-0.084918,-0.273486,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
knn = KNeighborsClassifier()

knn.fit(X_train_clean, y_train)

KNeighborsClassifier()

In [64]:
knn.score(X_train_clean, y_train)

0.2052015519168804

In [65]:
knn.score(X_test_clean, y_test)

0.018850554557011966

#### Decision Tree

In [66]:
dt = DecisionTreeClassifier(random_state=42)

dt.fit(X_train_clean, y_train)

DecisionTreeClassifier(random_state=42)

In [67]:
dt.score(X_train_clean, y_train)

0.772177286775827

In [68]:
dt.score(X_test_clean, y_test)

0.22975757310069703

### No EU

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X_no_eu, y_no_eu, test_size=0.2, 
                                                    random_state=42)

In [70]:
X_train_nums = X_train[['Date_year', 'Date_month', 'Value']]

ss = StandardScaler()

ss.fit(X_train_nums)
nums_df = pd.DataFrame(ss.transform(X_train_nums),
                      index=X_train_nums.index)

In [71]:
X_train_cat = X_train.select_dtypes('object')

ohe = OneHotEncoder(
    drop='first',
    sparse=False)

dums = ohe.fit_transform(X_train_cat)
dums_df = pd.DataFrame(dums,
                       columns=ohe.get_feature_names(),
                       index=X_train_cat.index)

In [72]:
X_train_clean = pd.concat([nums_df, dums_df], axis=1)

In [73]:
X_train_clean.columns

Index([                                         0,
                                                1,
                                                2,
                                        'x0_Asia',
                                      'x0_Europe',
                 'x0_Latin America and Caribbean',
                               'x0_North America',
                                     'x0_Oceania',
                      'x1_Electricity generation',
                         'x1_Electricity imports',
                          'x1_Electricity prices',
                      'x1_Power sector emissions',
                               'x2_CO2 intensity',
                 'x2_Day-ahead electricity price',
                                      'x2_Demand',
                         'x2_Electricity imports',
                                        'x2_Fuel',
                                       'x2_Total',
                               'x3_CO2 intensity',
                               

#### Preparing the Test Set

In [74]:
X_test_cat = X_test.select_dtypes('object')

test_dums = ohe.transform(X_test_cat)
test_dums_df = pd.DataFrame(test_dums,
                       columns=ohe.get_feature_names(),
                      index=X_test_cat.index)

In [75]:
X_test_nums = X_test[['Date_year', 'Date_month', 'Value']]

test_nums = ss.transform(X_test_nums)
test_nums_df = pd.DataFrame(test_nums,
                           index=X_test_nums.index)

In [76]:
X_test_clean = pd.concat([test_nums_df,
                 test_dums_df], axis=1)

In [77]:
X_test_clean.head()

,0,1,2,x0_Asia,x0_Europe,x0_Latin America and Caribbean,x0_North America,x0_Oceania,x1_Electricity generation,x1_Electricity imports,...,x3_Net Imports,x3_Nuclear,x3_Other Fossil,x3_Other Renewables,x3_Renewables,x3_Solar,x3_Total Generation,x3_Total emissions,x3_Wind,x3_Wind and Solar
97839,-1.387335,1.062195,-0.121495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30684,-1.387335,0.194096,-0.266708,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73356,0.715557,-1.252737,-0.259105,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
305580,0.084690,-0.384637,0.004771,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
303847,1.136136,-0.963370,-0.280544,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
knn = KNeighborsClassifier()

knn.fit(X_train_clean, y_train)

KNeighborsClassifier()

In [79]:
knn.score(X_train_clean, y_train)

0.5164080386670058

In [80]:
knn.score(X_test_clean, y_test)

0.2958896773329763

#### Decision Tree

In [81]:
dt = DecisionTreeClassifier(random_state=42)

dt.fit(X_train_clean, y_train)

DecisionTreeClassifier(random_state=42)

In [82]:
dt.score(X_train_clean, y_train)

0.8586070237919908

In [83]:
dt.score(X_test_clean, y_test)

0.6370062926763957

In [86]:
df_eu.Area.value_counts()

Estonia        4641
Portugal       4641
Slovakia       4641
Greece         4641
Denmark        4641
Sweden         4641
Lithuania      4641
France         4641
Austria        4641
Hungary        4641
Belgium        4641
Latvia         4641
Romania        4641
Spain          4641
Czechia        4641
Poland         4631
Bulgaria       4620
Ireland        4587
Netherlands    4541
Germany        4368
Slovenia       4041
Italy          4041
Finland        3436
Cyprus         3250
Luxembourg     2741
Croatia        2208
Malta          1974
Name: Area, dtype: int64

In [87]:
df_eu.head()

,Area,Country code,Date,Area type,Continent,Ember region,EU,OECD,G20,G7,Category,Subcategory,Variable,Unit,Value,Date_year,Date_month
25134,Austria,AUT,2015-01-01,Country,Europe,Europe,1.0,1.0,0.0,0.0,Electricity demand,Demand,Demand,TWh,5.88,2015.0,1.0
25135,Austria,AUT,2015-01-01,Country,Europe,Europe,1.0,1.0,0.0,0.0,Electricity generation,Aggregate fuel,Clean,%,63.54,2015.0,1.0
25136,Austria,AUT,2015-01-01,Country,Europe,Europe,1.0,1.0,0.0,0.0,Electricity generation,Aggregate fuel,Fossil,%,36.47,2015.0,1.0
25137,Austria,AUT,2015-01-01,Country,Europe,Europe,1.0,1.0,0.0,0.0,Electricity generation,Aggregate fuel,Gas and Other Fossil,%,30.79,2015.0,1.0
25138,Austria,AUT,2015-01-01,Country,Europe,Europe,1.0,1.0,0.0,0.0,Electricity generation,Aggregate fuel,"Hydro, Bioenergy and Other Renewables",%,51.31,2015.0,1.0


In [88]:
df_eu.loc[df_eu['Variable'] == 'Demand'].Value.min()

0.14

In [89]:
df_eu.loc[df_eu['Variable'] == 'Demand'].Value.max()

56.95

In [92]:
df_eu.loc[(df_eu['Variable'] == 'Demand')&(df_eu['Value'] == 0.14)]

,Area,Country code,Date,Area type,Continent,Ember region,EU,OECD,G20,G7,Category,Subcategory,Variable,Unit,Value,Date_year,Date_month
177941,Malta,MLT,2020-04-01,Country,Europe,Europe,1.0,0.0,0.0,0.0,Electricity demand,Demand,Demand,TWh,0.14,2020.0,4.0


In [93]:
df_eu.loc[(df_eu['Variable'] == 'Demand')&(df_eu['Value'] == 56.95)]

,Area,Country code,Date,Area type,Continent,Ember region,EU,OECD,G20,G7,Category,Subcategory,Variable,Unit,Value,Date_year,Date_month
124844,France,FRA,2017-01-01,Country,Europe,Europe,1.0,1.0,1.0,1.0,Electricity demand,Demand,Demand,TWh,56.95,2017.0,1.0


In [94]:
df_no_eu.loc[df_no_eu['Variable'] == 'Demand'].Value.min()

0.01

In [95]:
df_no_eu.loc[df_no_eu['Variable'] == 'Demand'].Value.max()

787.07

In [98]:
df_no_eu.loc[(df_no_eu['Variable'] == 'Demand')&(df_no_eu['Value'] == 0.01)]

,Area,Country code,Date,Area type,Continent,Ember region,EU,OECD,G20,G7,Category,Subcategory,Variable,Unit,Value,Date_year,Date_month
63622,Burundi,BDI,2021-07-01,Country,Africa,Africa,0.0,0.0,0.0,0.0,Electricity demand,Demand,Demand,TWh,0.01,2021.0,7.0


In [97]:
df_no_eu.loc[(df_no_eu['Variable'] == 'Demand')&(df_no_eu['Value'] == 787.07)]

,Area,Country code,Date,Area type,Continent,Ember region,EU,OECD,G20,G7,Category,Subcategory,Variable,Unit,Value,Date_year,Date_month
78410,China,CHN,2021-07-01,Country,Asia,Asia,0.0,0.0,1.0,0.0,Electricity demand,Demand,Demand,TWh,787.07,2021.0,7.0


### OECD Member States


In [99]:
filter_OECD = df['OECD'] == 1

In [100]:
df_OECD = df[filter_OECD]

In [101]:
relevant_col = ['Date_year', 'Date_month','Ember region',
                'Category', 'Subcategory','Variable',
                'Value']

In [102]:
X_OECD = df_OECD[relevant_col]
y_OECD = df_OECD['Area']

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X_OECD, y_OECD, test_size=0.2, random_state=42)

In [104]:
X_train_nums = X_train[['Date_year', 'Date_month', 'Value']]

ss = StandardScaler()

ss.fit(X_train_nums)
nums_df = pd.DataFrame(ss.transform(X_train_nums),
                      index=X_train_nums.index)

In [105]:
X_train_cat = X_train.select_dtypes('object')

ohe = OneHotEncoder(
    drop='first',
    sparse=False)

dums = ohe.fit_transform(X_train_cat)
dums_df = pd.DataFrame(dums,
                       columns=ohe.get_feature_names(),
                       index=X_train_cat.index)

In [106]:
X_train_clean = pd.concat([nums_df, dums_df], axis=1)

In [107]:
X_train_clean.columns

Index([                                         0,
                                                1,
                                                2,
                                      'x0_Europe',
                 'x0_Latin America and Caribbean',
                               'x0_North America',
                                     'x0_Oceania',
                      'x1_Electricity generation',
                         'x1_Electricity imports',
                          'x1_Electricity prices',
                      'x1_Power sector emissions',
                               'x2_CO2 intensity',
                 'x2_Day-ahead electricity price',
                                      'x2_Demand',
                         'x2_Electricity imports',
                                        'x2_Fuel',
                                       'x2_Total',
                               'x3_CO2 intensity',
                                       'x3_Clean',
                               

#### Preparing the Test Set

In [108]:
X_test_cat = X_test.select_dtypes('object')

test_dums = ohe.transform(X_test_cat)
test_dums_df = pd.DataFrame(test_dums,
                       columns=ohe.get_feature_names(),
                      index=X_test_cat.index)

In [109]:
X_test_nums = X_test[['Date_year', 'Date_month', 'Value']]

test_nums = ss.transform(X_test_nums)
test_nums_df = pd.DataFrame(test_nums,
                           index=X_test_nums.index)

In [110]:
X_test_clean = pd.concat([test_nums_df,
                 test_dums_df], axis=1)

In [111]:
X_test_clean.head()

,0,1,2,x0_Europe,x0_Latin America and Caribbean,x0_North America,x0_Oceania,x1_Electricity generation,x1_Electricity imports,x1_Electricity prices,...,x3_Net Imports,x3_Nuclear,x3_Other Fossil,x3_Other Renewables,x3_Renewables,x3_Solar,x3_Total Generation,x3_Total emissions,x3_Wind,x3_Wind and Solar
157639,0.887308,0.779279,-0.301368,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147090,0.887308,1.651986,-0.293457,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
262412,0.657512,1.361083,-0.295435,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71446,-0.261671,0.779279,-0.284887,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
179937,0.657512,-0.966136,-0.253901,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [112]:
knn = KNeighborsClassifier()

knn.fit(X_train_clean, y_train)

KNeighborsClassifier()

In [113]:
knn.score(X_train_clean, y_train)

0.4164013410770045

In [114]:
knn.score(X_test_clean, y_test)

0.23734118946786964

#### Decision Tree

In [115]:
dt = DecisionTreeClassifier(random_state=42)

dt.fit(X_train_clean, y_train)

DecisionTreeClassifier(random_state=42)

In [116]:
dt.score(X_train_clean, y_train)

0.8481468126404947

In [117]:
dt.score(X_test_clean, y_test)

0.4404345424415393